In [103]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import databento as db

In [73]:
def db_preparing(path: str) -> pd.DataFrame:
    store = db.DBNStore.from_file(path)
    df = store.to_df()
    df = df.reset_index(drop=True)
    trades = df[['ts_event', 'side', 'depth', 'price', 'size']]
    trades['ts_event'] = pd.to_datetime(trades['ts_event'].astype('int64'))
    trades = trades.set_index('ts_event')
    return trades

In [97]:
def resampling(path: str) -> pd.DataFrame:
    trades = db_preparing(path)
    sides = ['A', 'B', 'N']
    res_trds = []
    for side in sides:
        side_trades = trades[trades['side'] == f'{side}']
        resampled_trades = side_trades.resample("100ms").agg({"price": "mean", "size": "sum", "depth": "max"}).fillna(0)
        res_trds.append(resampled_trades['price'])
        res_trds.append(resampled_trades['size'])
    resampled_depth = trades['depth'].resample("100ms").max().fillna(0)
    res_trds.append(resampled_depth)
    trades_resampled = pd.DataFrame(res_trds).T
    trades_resampled = trades_resampled.fillna(0)
    trades_resampled.columns = ['trade_price_A', 'trade_volume_A', 'trade_price_B', 'trade_volume_B', 'trade_price_N', 'trade_volume_N', 'depth']
    return trades_resampled

In [ ]:
# path_trades = r'C:\Users\Эвелина Новикова\Downloads\xnas-itch-20250701.trades.FAST.dbn.zst'
# trades_resampled = resampling(path_trades)